In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
#from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
#from sklearn.datasets import make_circles
from sklearn.utils import shuffle
import sklearn
import itertools
import random
from tensorflow.keras.datasets import imdb
#from keras.preprocessing import image
from keras.preprocessing import sequence
import os
from google.colab import files
#from keras.preprocessing.image import ImageDataGenerator

def normalizeMinMax(x):
  minimum = tf.math.reduce_min(x,axis=0)
  maximum = tf.math.reduce_max(x,axis=0)
  return (x - minimum)/(maximum - minimum)

def plotting_model(train_data=[],train_labels=[],test_data=[],test_labels=[],prediction1=[],prediction2=[],prediction3=[]):
    plt.figure(figsize=(10,7))
    plt.scatter(train_data, train_labels, c="b", label="Training_data")
    plt.scatter(test_data, test_labels, c='r', label="Testing_data")
    plt.scatter(test_data, prediction1, c='g', label="Predicted_data_model1")
    plt.scatter(test_data, prediction2, c='y', label="Predicted_data_model2")
    plt.scatter(test_data, prediction3, c='c', label="Predicted_data_model3")

def plot_decision_boundary(model,X,y):
  """
  plots the decision boundary for a model classifying features (X) into classes
  """
  #define the axis boundaries of the meshgrid
  x_min,x_max = X[:,0].min() - 0.1, X[:,0].max() + 0.1
  y_min,y_max = X[:,1].min() - 0.1, X[:,1].max() + 0.1
  xx, yy = np.meshgrid(np.linspace(x_min,x_max,100),np.linspace(y_min,y_max,100))
  
  #create x value
  x_in = np.c_[xx.ravel(),yy.ravel()]
  y_pred = model.predict(x_in)
  if y_pred[0] > 1:
    print("doing multiclass classfication")
    y_pred = np.argmax(y_pred,axis=1).reshape(xx.shape)
  else:
    print("doing binary classification")
    y_pred = np.round(y_pred).reshape(xx.shape)
  #plot the decision boundary
  plt.contourf(xx,yy,y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
  plt.scatter(X[:,0],X[:,1],c='r',s=40,cmap=plt.cm.RdYlBu)
  plt.xlim(xx.min(),xx.max())
  plt.ylim(yy.min(),yy.max())

def pretty_confusion_matrix(y_test,y_pred,classes=None,textsize=20):
  
  cm = sklearn.metrics.confusion_matrix(y_test,tf.round(y_pred))
  cm_norm = cm.astype("float")/cm.sum(axis=1)[:,np.newaxis] #normalize the confusion matrix
  n_classes = cm.shape[0] #number of classes

  #prettifying it.
  fig, ax = plt.subplots(figsize = (25,25))
  #create a matrix plot
  cax = ax.matshow(cm,cmap=plt.cm.Blues)
  fig.colorbar(cax)

  #create classes

  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  #Label the axes
  ax.set(title="confusion matrix",xlabel="predicted label",ylabel="true label",
         xticks=np.arange(n_classes),yticks=np.arange(n_classes),
         xticklabels = labels, yticklabels=labels)
  
  #set x-axis labels to bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  #Adjust label 
  ax.yaxis.label.set_size(textsize)
  ax.xaxis.label.set_size(textsize)
  ax.title.set_size(textsize)

  #set threshold for different colors
  threshold = (cm.max() + cm.min())/2.0

  #plot text on each cell
  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i,f"{cm[i,j]} ({cm_norm[i,j]*100:.1f}%)",
             horizontalalignment="center",
             color="white" if cm[i,j]>threshold else "black", size=15)

def plot_random_data(train_data,train_labels,class_names):
  #plt.figure(figsize=(10,10))
  for i in range(1):
    #ax = plt.subplot(1,1,i+1)
    index = random.choice(range(100))
    inner_index = int(train_labels[index])
    #plt.imshow(train_data[index],cmap=plt.cm.binary)
    print("true class: ",class_names[inner_index])
    #plt.axis(False)

def test_random_data(test_data,test_labels,class_names,y_pred1):
    #plt.figure(figsize=(15,15))
    index = random.choice(range(len(test_data)))
    #plt.imshow(test_data[index],cmap=plt.cm.binary)
    inner_index = [int(test_labels[index]),int(test_labels[index+1]),int(test_labels[index+2]),int(test_labels[index+3]),int(test_labels[index+4]),int(test_labels[index+5]),int(test_labels[index+6]),int(test_labels[index+7]),int(test_labels[index+8]),int(test_labels[index+9])]
    print("true classes: ",class_names[inner_index[0]],class_names[inner_index[1]],class_names[inner_index[2]],class_names[inner_index[3]],class_names[inner_index[4]],class_names[inner_index[5]],class_names[inner_index[6]],class_names[inner_index[7]],class_names[inner_index[8]],class_names[inner_index[9]])
    #print(" predicted classes: ",class_names[tf.argmax(y_pred1[inner_index[0]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[1]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[2]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[3]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[4]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[5]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[6]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[7]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[8]]).numpy()],class_names[tf.argmax(y_pred1[inner_index[9]]).numpy()])
    print(" predicted classes: ",class_names[y_pred1[inner_index[0]]],class_names[y_pred1[inner_index[1]]],class_names[y_pred1[inner_index[2]]],class_names[y_pred1[inner_index[3]]],class_names[y_pred1[inner_index[4]]],class_names[y_pred1[inner_index[5]]],class_names[y_pred1[inner_index[6]]],class_names[y_pred1[inner_index[7]]],class_names[y_pred1[inner_index[8]]],class_names[y_pred1[inner_index[9]]])
    #plt.axis(False)

def image_augmentation(train_data,train_labels,width,height,channels,size):
  #creates a data generator object that transforms the image
  datagen = ImageDataGenerator(rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,
                                shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
  #pick a random image to transform
  for j in range(size):
    index = random.choice(range(len(train_data)))
    test_im = train_data[index]
    img = image.img_to_array(test_im)
    img = img.reshape((1,)+img.shape)
    
    i=0
    for batch in datagen.flow(img,save_prefix="test",save_format="jpeg"):
      #plt.figure(i)
      #plot = plt.imshow(tf.cast(image.img_to_array(batch[0]),dtype=tf.int32))
      print(j," ", i,"\n")
      casted_img = tf.cast(image.img_to_array(batch[0]),dtype=tf.int32)
      casted_img = tf.reshape(casted_img,shape=[1,width,height,channels])
      train_data = tf.concat(values=[train_data,casted_img],axis=0)
      reshaped_label = tf.reshape(train_labels[index],shape=[1,1])
      train_labels = tf.concat(values=[train_labels,reshaped_label],axis=0)
      i += 1
      if i > 4:
        break
      #plt.show()
  return train_data,train_labels

def test_manual(text,model,max,class_names):
  word_index = imdb.get_word_index()#retrieve the imdb dictionary
  #print(word_index)
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(input_text=text,filters='!$%().*+-/<=>?@[\\]^_{|}~\t\n',split=' ')#convert the input text into a sequence
  #print(tokens)
  new_token = [word_index[word] if word in word_index else 0 for word in tokens] #list comprehension of sequence to encode numerically according to imdb dictionary
  #print(new_token)
  padded_seq = sequence.pad_sequences(sequences=[new_token],maxlen=max)
  y_pred1 = model.predict(padded_seq)
  y_pred1 = tf.squeeze(tf.cast(tf.math.round(y_pred1),dtype=tf.int32))#for binary classification models
  print(" predicted classes: ",class_names[y_pred1])
  return padded_seq[0]

def decode_text(padded_seq):
  word_index = imdb.get_word_index()#retrieve the imdb dictionary
  reverse_word_index = {value:key for (key,value) in word_index.items()}#reverse the imdb dictionary
  #print(reverse_word_index)
  PAD = 0
  text = ""
  #print(padded_seq.shape)
  for x in padded_seq: 
    if x != PAD:
      text += reverse_word_index[x] +  " "
  print(text[:-1])

def split_input(text1):
  input_text = text1[:-1]
  output_text = text1[1:]
  return input_text,output_text


In [ ]:
#step 0: setting up (Preprocessing - Turn data into tensors) the data

path_to_file = tf.keras.utils.get_file('shakespear.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#read and decode to python readable format
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
print(len(text))
print(text[:250])

#create a sorted list of characters in ascending order
vocabulary = sorted(set(text)) #convert the full text into a character set (sequence) and then arrange in ascending order
#enumerate the character set with integers as a nested list with tuples.
vocab_enum = list(enumerate(vocabulary))
#create a dictionary with integer as index
int2char = {x:y for x,y in vocab_enum}
print(int2char.items())
#create a dictionary with character as index
char2int = {y:x for x,y in int2char.items()}
print(char2int.items())

def text_to_int(text):
  return np.array([char2int[c] for c in text])

print(text_to_int("I love you"))

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  finally:
    tuple1 = (int2char[x] for x in ints)
  return "".join(tuple1)

print(int_to_text(text_to_int("I love you")))

print(len(vocab_enum))
VOCAB_SIZE = len(vocabulary)  #number of vocabulary words.
SEQ_LEN = 100      #Maximum sequence length.


#Encode our whole dataset 
dataset = tf.data.Dataset.from_tensor_slices(text_to_int(text))

#divide the dataset into batches of sequence length
sequences = dataset.batch(SEQ_LEN+1,drop_remainder=True)
combined_data = sequences.map(split_input)

BATCH_SIZE =64
BUFFER_SIZE = 10000

for x,y in combined_data.take(1):
  print("INPUT: ",int_to_text(x),"\n")
  print("OUTPUt: ",int_to_text(y),"\n")

shuffled_combined_data = (combined_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))

print(shuffled_combined_data)
"""
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE)


#resize the sequence to common length. If greater than max size; trim. otherwise, pad.
resized_train_data = tf.cast(sequence.pad_sequences(sequences=train_data,maxlen=MAX_LEN,padding='pre',truncating='pre'),dtype=tf.int32)
resized_test_data = tf.cast(sequence.pad_sequences(sequences=test_data,maxlen=MAX_LEN,padding='pre',truncating='pre'),dtype=tf.int32)
print(len(resized_train_data[0]))
print(len(resized_test_data[0]))

"""
"""
#augmentation of images and addition to the training sets
augmented_train_data, augmented_train_labels = image_augmentation(resized_train_data,train_labels,width=width,height=height,channels=channels,size=500)
#print(train_data[0].shape,train_labels[0].shape)
#print(len(augmented_train_data),len(augmented_train_labels))


#shuffle the training data and labels
shuffled_aug_train_data, shuffled_aug_train_labels = shuffle(np.array(augmented_train_data),np.array(augmented_train_labels))
print(shuffled_aug_train_data[1],shuffled_aug_train_labels[1])
print(len(shuffled_aug_train_data),len(shuffled_aug_train_labels))

"""
"""
#convert class names into human readable format
class_names = ["Negative","Positive"]
plot_random_data(train_data,train_labels,class_names)


X_train,y_train,X_test,y_test = resized_train_data,train_labels,resized_test_data,test_labels

#Trick-Normalize data; but don't normalize the labels:if possible one-hot encode the labels
#X_train,y_train,X_test,y_test = normalizeMinMax(resized_train_data),train_labels,normalizeMinMax(resized_test_data),test_labels
print(len(X_train),len(y_train))
print(X_train[0])
print(y_train[0:25])
"""


1122304/1115394 [==============================] - 0s 0us/step
1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

dict_items([(0, '\n'), (1, ' '), (2, '!'), (3, '$'), (4, '&'), (5, "'"), (6, ','), (7, '-'), (8, '.'), (9, '3'), (10, ':'), (11, ';'), (12, '?'), (13, 'A'), (14, 'B'), (15, 'C'), (16, 'D'), (17, 'E'), (18, 'F'), (19, 'G'), (20, 'H'), (21, 'I'), (22, 'J'), (23, 'K'), (24, 'L'), (25, 'M'), (26, 'N'), (27, 'O'), (28, 'P'), (29, 'Q'), (30, 'R'), (31, 'S'), (32, 'T'), (33, 'U'), (34, 'V'), (35, 'W'), (36, 'X'), (37, 'Y'), (38, 'Z'), (39, 'a'), (40, 'b'), (41, 'c'), (42, 'd'), (43, 'e'), (44, 'f'), (45, 'g'), (46, 'h'), (47, 'i'), (48, 'j'), (49, 'k'), (50, 'l'), (51, 'm'), (52, 'n'), (53, 'o'), (54, 'p'), (55, 'q'), (56, 'r'), (57, 's'), (58, 't'), (59, 'u'), (60, 'v'), (61,

'\n#convert class names into human readable format\nclass_names = ["Negative","Positive"]\nplot_random_data(train_data,train_labels,class_names)\n\n\nX_train,y_train,X_test,y_test = resized_train_data,train_labels,resized_test_data,test_labels\n\n#Trick-Normalize data; but don\'t normalize the labels:if possible one-hot encode the labels\n#X_train,y_train,X_test,y_test = normalizeMinMax(resized_train_data),train_labels,normalizeMinMax(resized_test_data),test_labels\nprint(len(X_train),len(y_train))\nprint(X_train[0])\nprint(y_train[0:25])\n'

In [ ]:
#step 1: Creating the models- Define input layer, hidden layers and output layer
RNN_UNITS = 1024
EMBEDDING_DIM = 256

tf.random.set_seed(40)
model1 = tf.keras.Sequential(name='RNN1')
model1.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim=EMBEDDING_DIM,batch_input_shape=[BATCH_SIZE,None],name="Embedding_layer"))#input_dim: maximum size of the vocabulary, this convert positive integer indexes to a vector
model1.add(tf.keras.layers.LSTM(units=RNN_UNITS,return_sequences=True,stateful=True,activation='tanh',recurrent_initializer='glorot_uniform',recurrent_activation='sigmoid'))#LSTM layer which return sequences:return the last output for each time step, last state of a batch will be used as the initial state of the next batch
model1.add(tf.keras.layers.Dense(units=256,activation='relu',name='hidden_dense_layer1'))
model1.add(tf.keras.layers.Dense(units=VOCAB_SIZE,name='output_layer'))
"""
model2 = tf.keras.Sequential(name='neural_network2')
model2.add(tf.keras.layers.Dense(100, activation='relu',name='Hidden_layer1'))  # Hidden layer 1 with 100 neurons
model2.add(tf.keras.layers.Dense(100, activation='relu', name='Hidden_layer2'))  # Hidden layer 2 with 100 neurons
model2.add(tf.keras.layers.Dense(1, name='output_layer'))  # output layer

model3 = tf.keras.Sequential(name='neural_network3')
model3.add(tf.keras.layers.Dense(50,activation='relu',name='Hidden_layer1')) #Hidden layer 1 with 10 neurons
model3.add(tf.keras.layers.Dense(100,activation='relu',name='Hidden_layer2')) #Hidden layer 2 with 100 neurons
model3.add(tf.keras.layers.Dense(500,activation='relu',name='Hidden_layer3')) #Hidden layer 3 with 500 neurons
model3.add(tf.keras.layers.Dense(100, activation='relu', name='Hidden_layer4'))#Hidden layer 4 with 100 neurons
model3.add(tf.keras.layers.Dense(10, activation='relu', name='Hidden_layer5'))#Hidden layer 5 with 10 neurons
model3.add(tf.keras.layers.Dense(1,name='output_layer')) #output layer
"""

"\nmodel2 = tf.keras.Sequential(name='neural_network2')\nmodel2.add(tf.keras.layers.Dense(100, activation='relu',name='Hidden_layer1'))  # Hidden layer 1 with 100 neurons\nmodel2.add(tf.keras.layers.Dense(100, activation='relu', name='Hidden_layer2'))  # Hidden layer 2 with 100 neurons\nmodel2.add(tf.keras.layers.Dense(1, name='output_layer'))  # output layer\n\nmodel3 = tf.keras.Sequential(name='neural_network3')\nmodel3.add(tf.keras.layers.Dense(50,activation='relu',name='Hidden_layer1')) #Hidden layer 1 with 10 neurons\nmodel3.add(tf.keras.layers.Dense(100,activation='relu',name='Hidden_layer2')) #Hidden layer 2 with 100 neurons\nmodel3.add(tf.keras.layers.Dense(500,activation='relu',name='Hidden_layer3')) #Hidden layer 3 with 500 neurons\nmodel3.add(tf.keras.layers.Dense(100, activation='relu', name='Hidden_layer4'))#Hidden layer 4 with 100 neurons\nmodel3.add(tf.keras.layers.Dense(10, activation='relu', name='Hidden_layer5'))#Hidden layer 5 with 10 neurons\nmodel3.add(tf.keras.lay

In [ ]:
#try the model
for input_example_batch, target_example_batch in shuffled_combined_data.take(1):
    example_batch_predictions = model1(input_example_batch)
    print(input_example_batch.shape)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model1.summary()
print(example_batch_predictions[0])
print("sequence_length,vocab_size",example_batch_predictions[0].shape)

#take one random sample out of 65 logits (A logit is an input to a softmax function)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices)
print("sequence_length",sampled_indices.shape) 
#remove the extra size 1 dimension
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
print(sampled_indices)
print("squeezed sample_indices",sampled_indices.shape) 

print("Input:\n", int_to_text(input_example_batch[0]))
print("Next Char Predictions:\n", int_to_text(sampled_indices))


(64, 100)
(64, 100, 65) # (batch_size, sequence_length, vocab_size)
Model: "RNN1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_layer (Embedding)  (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
hidden_dense_layer1 (Dense)  (64, None, 256)           262400    
_________________________________________________________________
output_layer (Dense)         (64, None, 65)            16705     
Total params: 5,542,721
Trainable params: 5,542,721
Non-trainable params: 0
_________________________________________________________________
tf.Tensor(
[[-0.00015927 -0.00075921  0.0011484  ...  0.00407171  0.00325431
   0.00345966]
 [-0.0040196   0.00108111  0.00278038 ...  0.0050939   0.00048026
   0.00442758]
 [-0.0032

In [ ]:
#step 2: Compiling the  - Define the loss function and optimizer for NNs
model1.trainable = True
model1.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
#model2.compile(loss=tf.keras.losses.mae,optimizer=tf.keras.optimizers.Adam(),metrics=['mae'])
#model3.compile(loss=tf.keras.losses.mae, optimizer=tf.keras.optimizers.Adam(),metrics=['mae'])

#step 3: Fitting (Tranining) the models
#A callback works during model training.  
Earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience = 3)
learning_rate_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch:1e-3 * 10 **(epoch/7))#start with a low training rate and increase with number of epochs
#directory for checkpoint saving
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

#training_curve1 = model1.fit(X_train,y_train,epochs=20,verbose=1,callbacks=[Earlystop_callback,learning_rate_scheduler_callback])
training_curve1 = model1.fit(shuffled_combined_data,epochs=1,verbose=1,callbacks=[Earlystop_callback,checkpoint_callback])
#training_curve2 = model2.fit(X_train_Normalized,y_train,epochs=500,verbose=0,callbacks=[Earlystop_callback,learning_rate_scheduler_callback])
#training_curve3 = model3.fit(X_train_Normalized, y_train,epochs=500,verbose=0callbacks=[Earlystop_callback,learning_rate_scheduler_callback])


In [ ]:
#step 4: Evaluating the models
def generate_text(start_string):
  #number of characters to generate
  num_generate = 2

  #converting start string to numbers
  input_eval = np.array([char2int[c] for c in start_string])
  print(input_eval,"\n")
  #empty string to store results
  text_generated = ""
  temperature = 1.0
  BATCH_SIZE =64
  BUFFER_SIZE = 10000

  #reset the model's states
  for i in range(num_generate):
    dataset = tf.data.Dataset.from_tensor_slices(input_eval)
    new_dataset = dataset.batch(SEQ_LEN+1,drop_remainder=True)
    latest_dataset = new_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    print(i)
    for input_example_batch in latest_dataset.take(1):
      predictions = model1(input_example_batch)
      print(input_example_batch.shape,"\n")
      print(predictions.shape, "# (batch_size, sequence_length, vocab_size)",'\n')
      print(i)
      """
      #remove the batch dimension
      #predictions = tf.squeeze(predictions,axis=0)

      #using a categorical distribution to predict the character returned by the model
      predictions = predictions/temperature
      predicted_id = tf.random.categorical(predictions[0],num_samples=1) #last character id is obtained

      #pass this predicted character as the input to the next iteration with previous state
      input_eval = predicted_id
      text_generated.append(int2char[predicted_id])
      #print(predicted_id,"\n")

      """
inp = "Because narrative paragraphs resemble fiction an untrue story, you have a little more freedom to write the story in the style you prefer. This is known as artistic freedom or artistic license. You can use the first person narrative style and include words that clearly refer to you I, me, my, mine, etc., or you can try to tell the story from a purely objective point of view that is not personal but gives a straight-forward, factual account of what happened. If your teacher asks you to write about a personal experience, try to tell it by using the first person. This is the easiest style in which to write something. You might choose something that you remember well or something that changed your life. Teachers who work with a large number of foreign born students often ask them to write about the time they first arrived in the United States. That is a good assignment because it allows you to write in the first person and the details in this kind of paragraph are likely to be very vivid. Here is an example"

generate_text(start_string=inp)





"""
model1.summary()

#model2.summary()
#model3.summary()
y_pred1 = model1.predict(X_test)
y_pred1 = tf.squeeze(tf.cast(tf.math.round(y_pred1),dtype=tf.int32))#for binary classification models
#y_pred2 = model2.predict(X_test_Normalized)
#y_pred3 = model3.predict(X_test_Normalized)
model1.evaluate(X_test,y_test)
#model2.evaluate(X_test_Normalized,y_test)
#model3.evaluate(X_test_Normalized, y_test)

#plotting_model(X_train, y_train, X_test, y_test, y_pred1, y_pred2, y_pred3)
plot_model(model=model1, show_shapes=True)
#plot_decision_boundary(model1,X_test,y_test)
test_random_data(test_data,test_labels,class_names,y_pred1)
#mae1 = tf.keras.metrics.mean_absolute_error(y_test,tf.argmax(y_pred1[0]).numpy())
#mse1 = tf.keras.metrics.mean_squared_error(y_test,tf.argmax(y_pred1[0]).numpy())
mae1 = tf.keras.metrics.mean_absolute_error(y_test,y_pred1[0])
mse1 = tf.keras.metrics.mean_squared_error(y_test,y_pred1[0])
print(mae1, mse1)
plt.figure(figsize=(10,7))
pd.DataFrame(training_curve1.history).plot()
plt.title("Training curve for model1")
plt.ylabel("loss")
plt.xlabel("epochs")

"""
"""
#plot the learning rate vs the loss to find the ideal learning rate for a given model
lrs = 1e-4 * 10 **((tf.range(25))/20)
plt.figure(figsize=(10,7))
plt.semilogx(lrs,training_curve1.history["loss"][:25])
plt.xlabel("learnig rate in log")
plt.ylabel("loss")
plt.title("loss vs learning rate")
"""
"""
#classification_report1 = sklearn.metrics.classification_report(y_test,tf.argmax(y_pred1,axis=1).numpy())
#confusion_matrix1 = sklearn.metrics.confusion_matrix(y_test,tf.argmax(y_pred1,axis=1).numpy())
classification_report1 = sklearn.metrics.classification_report(y_test,y_pred1)
confusion_matrix1 = sklearn.metrics.confusion_matrix(y_test,y_pred1)
print(classification_report1)
print(confusion_matrix1)
#pretty_confusion_matrix(y_test,tf.argmax(y_pred1,axis=1).numpy(),classes=class_names)
pretty_confusion_matrix(y_test,y_pred1,classes=class_names)

text1 = "This is indeed a nice movie. Character are good. I really love it"
test_manual(text=text1,model=model1,max=250,class_names=class_names)
"""
"""
#plot_model(model=model2, show_shapes=True)
mae2 = tf.keras.metrics.mean_absolute_error(y_test, tf.squeeze(y_pred2)).numpy()
mse2 = tf.keras.metrics.mean_squared_error(y_test, tf.squeeze(y_pred2)).numpy()
print(mae2, mse2)
pd.DataFrame(training_curve2.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")
#plot_model(model=model3, show_shapes=True)
mae3 = tf.keras.metrics.mean_absolute_error(y_test, tf.squeeze(y_pred3)).numpy()
mse3 = tf.keras.metrics.mean_squared_error(y_test, tf.squeeze(y_pred3)).numpy()
print(mae3, mse3)
pd.DataFrame(training_curve3.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")

#step 5: Comparing the models
model_results = [["model1",mae1,mse1],["model2",mae2,mse2],["model3",mae3,mse3]]
all_results = pd.DataFrame(model_results,columns=["Model_name", "MAE","MSE"])
print(all_results)

#step 6: Saving the models
#Saving can be done in one of SavedModel format or in HDF5 format.
model1.save("model1Saved.h5",save_format='HDF5')
model2.save("model2Saved", save_format='SavedModel')
model3.save("model3Saved", save_format='SavedModel')

#loading model (optional)
loadmodel1 = tf.keras.models.load_model("model1Saved.h5",custom_objects={"mae": tf.keras.losses.mae})
loadmodel1.summary()
loadmodel_pred = loadmodel1.predict(X_test)
original_pred = model1.predict(X_test)
print(loadmodel_pred == original_pred)
"""

